# 뉴스데이터 연관키워드 분석 (2020년 07월 01일 ~ 07월 05일) / 형태소분석 적용

In [52]:
%matplotlib inline

In [53]:
try:
    import pandas as pd
except:
    !pip install pandas
    !pip install numpy==1.19.0
    !pip install matplotlib
    import pandas as pd
    

import numpy as np
import matplotlib.pyplot as plt

import warnings

In [54]:
warnings.filterwarnings("ignore")

## 2. 키워드 추출

#### [2-1) 텍스트 데이터 전처리]

In [ ]:
import re

# 텍스트 정제 함수 : 할글 이외의 문자는 전부 제거
def text_cleaning(text):
#     hangul = re.compile('[^ㄱ-ㅎ|가-힣|a-z|A-Z|0-9|\*]') #한글의 정규표현식을 나타냅니다.
    hangul = re.compile('[^ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]') #한글의 정규표현식을 나타냅니다.
    result = hangul.sub(' ', text)
    result = re.sub(' +', ' ', result)
    return result



#### [2-2) KoNLPy를 이용한 키워드 추출 (형태소 분석기)]
#### 한국어 약식 불용어 사전 사용 (http://ranks.nl/stopwords/korean)

In [14]:
# csv파일 불러오기
df = pd.read_csv('news_data_ori_0701_0705.csv')
# df = pd.read_csv('tweet_data_ori_0925.csv')
df.head()

print(df)

df['ko_text'] = df['본문'].apply(lambda x: text_cleaning(x))
df.head()


#konlpy 문서 : https://konlpy-ko.readthedocs.io/ko/v0.4.3/api/konlpy.tag/
#형태소 분석 태그정리 : https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0
#사전 튜닝 : https://cromboltz.tistory.com/18

# from konlpy.tag import Okt
from eunjeon import Mecab
from collections import Counter
from IPython.display import clear_output


korean_stopwords_path = './korean_stopwords.txt'
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

nouns_cnt = 1
count_data = []

def get_nouns(x):
    global nouns_cnt, count_data
    
#     nouns_tagger = Okt()
    nouns_tagger=Mecab(dicpath='D:/DEV/python/workspace_python/python1/0.prototype/lib/mecab/mecab-ko-dic')
    nouns = nouns_tagger.nouns(x)
    count_data.extend([y for y in nouns])
#     count_data.extend([z for y in nouns for z in y])

    #한글자 키워드를 제거합니다.
    nouns = [noun for noun in nouns if len(noun) > 1]

    #불용어를 제거합니다.
    nouns = [noun for noun in nouns if noun not in stopwords]

    clear_output(wait=True)
    print(f'형태소,불용어 제거중 .. {round((nouns_cnt / df.shape[0])*100,3)} %')
    nouns_cnt+=1
    
    return nouns



                        생성일                                                 본문
0       2020-09-23 10:10:53  RT @M2MPD: 44초 짜리 티저 44시간째 돌려보는중..\n  ㄴ 이거 방금 ...
1       2020-09-23 10:10:52  RT @treasuremembers: 여러분...거울에 있는 저만 보세요!!!\n다...
2       2020-09-23 10:10:52  RT @bts_love_myself: 유엔 총회에 참여한 방탄소년단의 메시지를 아래...
3       2020-09-23 10:10:52  RT @iyagideul: [비터애플/BL] 제이윰 작가님의 &lt;여우, 꽃피다&...
4       2020-09-23 10:10:52  RT @hellokoook: 이 짤에서 못나가겠네... https://t.co/zC...
...                     ...                                                ...
337513  2020-09-25 07:06:29    @energydrink1215 @lastm2704 @SaintPorte 살려야 한다.
337514  2020-09-25 07:06:29  흑과 백, 선과 악, 빛과 어둠, 참과 거짓, 이슬람교와 그리스도교, 만남과 헤어짐...
337515  2020-09-25 07:06:28  @L4NYU 그거나쁘지않네요 재밌으면됐죠 지나가는사람들한테 너도술래잡기할래!!하면 ...
337516  2020-09-25 07:06:27  @eto__o 이제 곧 주말인데 모이도 놀아...(폭풍이 휩쓸고 간 자리에서 얼빠진...
337517  2020-09-25 07:06:26                           北  통지문에는\n우리국민 월북 시도 없었다

[337518 rows x 2 columns]


In [15]:
# df = df.iloc[:1000,:]


import time
start = time.time()  # 시작 시간 저장

nouns_cnt = 1
# df['nouns'] = df['ko_text'].apply(lambda x: get_nouns(x))
# df['nouns'] = df['본문'].apply(lambda x: get_nouns(x))
df['nouns'] = df['ko_text'].apply(lambda x: get_nouns(x))

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간


print(df.shape)
df.head()

형태소,불용어 제거중 .. 100.0 %
(1000, 3)


,생성일,본문,nouns
0,2020-09-23 10:10:53,RT @M2MPD: 44초 짜리 티저 44시간째 돌려보는중..\n ㄴ 이거 방금 ...,"[티저, 중, 거, 방금, 요]"
1,2020-09-23 10:10:52,RT @treasuremembers: 여러분...거울에 있는 저만 보세요!!!\n다...,"[거울, 건, 볼, 생각, 거울, 속, 사랑, 트, 레저, 메이커]"
2,2020-09-23 10:10:52,RT @bts_love_myself: 유엔 총회에 참여한 방탄소년단의 메시지를 아래...,"[유엔, 총회, 참여, 방탄소년단, 메시지, 아래, 링크, 확인]"
3,2020-09-23 10:10:52,"RT @iyagideul: [비터애플/BL] 제이윰 작가님의 &lt;여우, 꽃피다&...","[비터, 애플, 제이, 윰, 작가, 여우, 꽃피, 전, 권, 간, 기념, 이벤트, ..."
4,2020-09-23 10:10:52,RT @hellokoook: 이 짤에서 못나가겠네... https://t.co/zC...,[짤]


In [16]:
#빈도 수 구하기.

# count_data = [y for x in df['nouns'].tolist() for y in x]

from collections import Counter
count = Counter(count_data)

node_df = pd.DataFrame(count.items(), columns=['node', 'nodesize'])
# node_df = node_df[node_df['nodesize'] >= 50]

# 형태소 데이터 / csv파일로 저장
df.to_csv('save/tmp_mecab_data_growth2.csv', index=False)
# 카운트 데이터 / csv파일로 저장
node_df.to_csv('save/tmp_mecab_count_growth2.csv', index=False)

node_df.head()

,node,nodesize
0,티저,2
1,중,23
2,거,39
3,방금,3
4,요,22


In [55]:
from ast import literal_eval

# literal_eval이 기본 자료형만
# eval은 모든 스트링을 코드화

df = pd.read_csv('save/tmp_mecab_data2.csv',  converters={'nouns': literal_eval})
# df = pd.read_csv('save/tmp_data2.csv', converters={'nouns': eval})
node_df = pd.read_csv('save/tmp_mecab_count2.csv')

## 3. 연관 키워드 추출하기

### [트위터 연관 키워드 분석]

In [56]:
transactions = df['nouns'].tolist()
transactions = [transaction for transaction in transactions if transaction] # 공백 문자열을 방지합니다.and
# transactions

In [57]:
try:
    from mlxtend.preprocessing import TransactionEncoder
except:
    !pip install mlxtend
    from mlxtend.preprocessing import TransactionEncoder

from mlxtend.frequent_patterns import apriori as apriori2
from mlxtend.frequent_patterns import association_rules

from mlxtend.frequent_patterns import fpgrowth

from mlxtend.frequent_patterns import fpmax



te = TransactionEncoder()
te_result = te.fit(transactions).transform(transactions)

print(te_result.shape[0]*te_result.shape[1])

1112790970


In [58]:
df2 = pd.DataFrame(te_result, columns=te.columns_)
df2.head()

,가가,가가와,가가와현,가감,가갸,가건물,가게,가격,가격대,가격제,...,힐러리,힐리,힐링,힐스,힐책,힐튼,힘겨루기,힘줄,힘찬,힙합
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# keyword = '손흥민'

df2 = pd.DataFrame(te_result, columns=te.columns_)

import time
start = time.time()  # 시작 시간 저장

# http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/fpgrowth/
#------------------------------------
# 공통정보
#------------------------------------
# News Cnt = 21,194
# Mecab + 1글자제거 + 불용어제거 = 190초
# Transaction Cnt = 1,112,790,970 (11억)

#------------------------------------
# max_len=(default: None) 1:다 (무제한 관계)
#------------------------------------
#-- Apriori --
# 0.05 = 32초 / (1,106개 / 2,526개)
# 0.03 = 251초 / (5,974개 / 44,564개)
# 0.02 =  초 / 개 (메모리 부족.)
# 0.01 =  초 / 개 (메모리 부족.)

#-- fpgrowth --
# 0.05 = 9초 (1,106개 / 2,526개)
# 0.03 = 24.52초 (5,974개 / 44,564개)
# 0.01 = 383초

#-- Apriori=Low_memory --
# 0.05 = 41초 (1,106개 / 2,526개)
# 0.03 = 141초 (5,974개 / 44,564개)
# 0.01 = 

#------------------------------------
# Apriori / max_len=2
#------------------------------------
# 0.05 = 22초 / 1,106개
# 0.03 = 72초 / 5,974개
# 0.01 = 1387초 / 75,812개

#------------------------------------
# fpgrowth / max_len=2 (교집합으로 제외시켜 나가는 구조)
#------------------------------------
# 0.05 = 12초 / 1,106개
# 0.03 = 27.58초 / 5,974개
# 0.01 = 117 초 / 75,812개 (Apriori와 검색결과 같음)
# 0.001 = 2580 초 / 1,895,362개
# 0.0009 = 2585 초 / 2,098,240개

#------------------------------------
# Apriori / max_len=2 + low_memory=True (Low_memory는 1:1분석에만 효율이 있다 / fpgrowth는 1:다 분석에 효율이 있다.)
#------------------------------------
# 0.05 = 18.49초 / 1,106개
# 0.03 = 25.86초 / 5,974개
# 0.01 = 41초 / 75,812개 (세부분석 : 0.49초)
# 0.001 = 85초 / 1,895,362개 (세부분석 : 11초)
# 0.0009 = 84.7초 / 2,098,240개 (세부분석 : 11초)
# 0.0001 = 186초 / 15,440,440개 (세부분석 : 83초)



#------------------------------------
# fpmax / max_len=2
#------------------------------------
# 0.01 = 106 초 / 41750 개  #분석 결과 다름 support_only=True


# itemset = apriori2(df2, min_support=0.0001, use_colnames=True, max_len=2, low_memory=True)
# itemset = fpgrowth(df2, min_support=0.001, use_colnames=True, max_len=2)
itemset = fpgrowth(df2, min_support=0.01, use_colnames=True)
# itemset = fpmax(df2, min_support=0.01, use_colnames=True, max_len=2)



print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

itemset.head()



In [ ]:
start = time.time()  # 시작 시간 저장
#연결규칙
itemset2 = association_rules(itemset, metric="confidence", min_threshold=0.0000001)
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
itemset2

In [ ]:
# 연관키워드 알고리즘 데이터 csv저장
itemset2.to_csv('save/tmp_mecab_last_growth2(0_001).csv', index=False)

In [32]:
# load파일
itemset2 = pd.read_csv('save//tmp_mecab_last_growth2(0_001).csv', converters={'antecedents': eval, 'consequents': eval})
itemset2.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(법무부),(감찰),NaN,NaN,0.010003,NaN,NaN,NaN,NaN
1,(감찰),(법무부),NaN,NaN,0.010003,NaN,NaN,NaN,NaN
2,(종로구),(서울),NaN,NaN,0.010663,NaN,NaN,NaN,NaN
3,(서울),(종로구),NaN,NaN,0.010663,NaN,NaN,NaN,NaN
4,(환자),(데시),NaN,NaN,0.010003,NaN,NaN,NaN,NaN


In [ ]:
# 조건이 키워드
keyword = '코로나'
# zz = itemset2[itemset2['antecedents'].apply(lambda x : (keyword in x))]

zz = itemset2[itemset2['antecedents'].apply(lambda x : (keyword in x) & (len(x) == 1))]
# zz = itemset2[itemset2['antecedents'].apply(lambda x : (len(x) == 1))]

zz = zz[zz['consequents'].apply(lambda x : (len(x) == 1))]

print(zz.shape)

#카운트 추가
node_df = pd.read_csv('save/tmp_mecab_count2.csv')
def getcount(data) :
    return node_df[node_df['node'].apply(lambda x : data == frozenset({x}))]['nodesize'].values[0]

def getcount_news(data) :
    return df2[next(iter(data))].sum()

# zz['출현_카운트'] = zz['consequents'].apply(lambda x : getcount(x))
# zz['뉴스_카운트'] = zz['consequents'].apply(lambda x : getcount_news(x))

# iat는 한개의 값만 사용할 수 있고 iloc나 loc는 슬라이싱 해서도 사용할 수 있다
# 슬라이싱 하면 결과가 문자값만 나오는게 아니라서 안예쁘지만

# node_df[node_df['node'].apply(lambda x : '의원' == x)]['nodesize'].iat[0]
# node_df[node_df['node'].apply(lambda x : '의원' == x)]['nodesize'].iloc[0]
# node_df[node_df['node'].apply(lambda x : '의원' == x)]['nodesize'].values[0]



pd.set_option('display.max_rows', 1000)
qq = zz.sort_values(by=['support','confidence','lift'], axis='index', ascending=False)

qq.head(20)

